# Abstract Base Classes and Operator Overloading

We often need to make a distinction between concrete classes that have a compete set of attributes and methods, and an abstract class that is missing some details.

This parallels the philosophical idea of abstraction as a way to summarize complexities.

We might say that a sailboat and an airplane have a common, abstract relationship of being vehicles, but the details of how they move are distinct.

We have two approaches to defining similar things:

**Duck Typing:**

When two class definitions have the same attributes and methods, then instances of the two classes have the same protocol and can be used interchangeably.

We often say, "when I see a bird that walks like a duck and swims like a duck and quacks like a duck, I call that bird a duck."

**Inheritance**

When two class definitions have common aspects, a subclass can share common features of a superclass.

The implementation details of the two classes should be interchangeable when we use the common features defined by superclass.

  



We can take inheritance one step further.

We can have a superclass definitions that are abstract: this means they are not directly usable by themselves, but can be used through inheritance to create concrete classes.

We have to acknowledge a terminology problem around the term base and superclass.

This is confusing because they are synonyms. There are two parallel metaphors here and we flip back and forth between them.



![Abstract](./abstract.png)


Our base class, named BaseClass, has a special class, abc.ABC, as a parent class.

This provides some special metaclass features that help make sure the concrete classes have replaced the abstractions.

The diagram shows an abstract method, `a_method`, which does not have a defined body.

A subclass must provide this.

# CREATING AN ABSTRACT BASE CLASS

Imagine we are creating a media player wtih third-party plugins.

It is advisable to create an abstract base class (ABC) in this case to document what API the third-party plugins should provide.

The greneral design is to have a common feature, like `play()`, that applies to a number of classes.

We do not want to pick some particular media format to use as a superclass; it seems somehow wrong to claim that some format is foundational and all others are derived from it.

We would prefer to define the media player as an abstraction.

Each unique kind of media file format can provide a concrete inmplementation of the abstraction.

The `àbc` module provides the tools to do this.


In [144]:
import abc

class MediaLoader(abc.ABC):
    
    @abc.abstractmethod
    def play(self) -> None:
        ...
    @property
    @abc.abstractmethod
    def ext(self) -> str:
        ...

`abc.ABC` class introduces a *metaclass* - a class used to build the concrete class definitions.

Python's default metaclass is `type`.

The default metaclass does not check for abstract methods when we try to create an instance.

The `abc.ABC` class includes an extension to the `type` metaclass to prevent us from creating instances of classes that are not fully defined.

Python uses decorators widely to make modifications to genral nature of the method or function.

In this case, it provides additional details used by the metaclass that was incldued by the `ABC` class.

Becuase we marked a method or property as abstract, any subclass of this class must implement that method or property in order to be useful, concrete implementation.

We've used the `@property` decorator on the `ext()` method, also. 

Our intent for the ext property is to provide a simple class-level variable with a string literal value. I

t's helpful to describe this as a `@property` to allow the implementation to choose between a simple variable and a method that implements the property. 

A simple variable in the concrete class will meet the expectations of the abstract class at runtime and will also help mypy to check the code for consistent use of types. 

A method could be used as an alternative to a simple attribute variable in case some more sophisticated computation is required.

One of the consequences of marking these properties is the class now has a new special attribute, `__abstractmethods__`. 

This attribute lists all of the names that need to be filled in to create a concrete class:

In [145]:
MediaLoader.__abstractmethods__

frozenset({'ext', 'play'})

See what happens if you implement a subclass? 

We'll look at an example that doesn't supply concrete implementations for the abstractions. 

We'll also look at an example that does supply the required attribute:

In [146]:
class Wav(MediaLoader):
    pass

In [147]:
x = Wav()

TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

In [160]:
class Ogg(MediaLoader):
    ext = '.ogg'
    
    def play(self):
        pass
    

In [161]:
o = Ogg()

The definition of a `Wav` subclass fails to implement either of the abstract attributes.

When we try to create an instance of the `Wav` class, an exception is raised.

Because this subclass of `MediaLoader` is still abstract, it is not possible to instantiate the class.

The class is still a potentially usefull abstract class, but you would have to subclass it and fill in the abstract placeholders before it can actually do anything.

The `Ogg` subclass supplies both attributes, so it can instantiate cleanly.

It is true, the body of the `play()` method does not do very much.

What is important is that all of the placeholders were filled, making `Ogg` a concrete subclass of the abstract ``MediaLoader` class.

 

Note that there is a subtle issue with using a class-level variable for the preferred media file extension.

Because the `ext` attribute is a varibal, it can be updated.

Using `o.ext = '.xyz'` is not expressly prohibited.

Python does not have an easy, obvious way to create read-only attributes.

We often rely on documentation to explain the consequences of changing the value of the `ext` attribute.

This has clear advantages when crating a complex application. The use of abstraction like this makes it very easy for mypy to conclude that a class does (does not) have the required methods and attributes.

This also mandates a certain amount of fussy importing to be sure that the module has access to the necessary abstract base classes for an application.

One of the advantages of duck typing is the ability to avoid complex imports and still create a useful class that can act polymorphic with peer classes.

This advantage is often outweighed by the ability of the `abc.ABC` class definition to support type checking and documentation.

the `abc.ABC` class also provides far more useful error messages when something is wrong.

One important use cas for the ABCs is the `collections` module.

This module defines the built-in generic collections using a sophisticated set of base classes and mixins.

# ABCs of Collections

A really comprehensive use of the abstract base classes in the Python standard library lives in the `collections` module.

The collections we use are extensions of the `Collection` abstract class.

`Collection` is an extension of an even more fundamental abstraction, `Container`.

Since the foundation is the `Container` class, let's inspect it in interpreter to see what method this class requires:


> import collections.abc as Container
> `Container.__abstractmethods__`

>> `frozenset({'__contains__'})`


So, the Container class has exactly one abstract method that needs to be implemented, `__contains__()`. 

You can issue `help(Container.__contains__)` to see what the function signature should look like:

`help(Container.__contains__)`
Help on function `__contains__` in module collections.abc:
`__contains__(self, x)`


We can see that `__contains__()` needs to take a single argument. 

Unfortunately, the help file doesn't tell us much about what that argument should be, but it's pretty obvious from the name of the ABC and the single method it implements that this argument is the value the user is checking to see whether the container holds.

This `__contains__()` special method implements the Python in operator. 

This method is implemented by set, list, str, tuple, and dict. 

However, we can also define a silly container that tells us whether a given value is in the set of odd integers:

In [162]:
from collections.abc import Container

class OddIntegers:
    
    def __contains__(self, x: int) -> bool:
        return x % 2 != 0

We've used the modulo test for oddity. If the remainder of x divided by two is zero, then x was even, otherwise x was odd.

Here's the interesting part: we can instantiate an `OddContainer` object and determine that, even though we did not extend `Container`, the class behaves as a `Container` object:

In [163]:
odd = OddIntegers()
isinstance(odd, Container)

True

In [164]:
issubclass(OddIntegers, Container)

True

And that is why duck typing is way more awesome than classical polymorphism. 

We can create is-a relationships without the overhead of writing the code to set up inheritance (or worse, multiple inheritance).

One cool thing about the Container ABC is that any class that implements it gets to use the `in` keyword for free. 

In fact, `in` is just syntax sugar that delegates to the `__contains__()` method. 

Any class that has a `__contains__()` method is a Container and can therefore be queried by the `in` keyword. For example:

In [165]:
odd = OddIntegers()

1 in odd

True

In [166]:
2 in odd


False

In [167]:
3 in odd

True

In [168]:
4 in odd

False

The real value here is the ability to create new kinds of collections that are completely compatible with Python's built-in generic collections. 

We could, for example, create a dictionary that uses a binary tree structure to retain keys instead of a hashed lookup. 

We'd start with the `Mapping` abstract base class definitions, but change the algorithms that support methods like `__getitem__()`, `__setitem__()`, and `__delitem__()`.

Python's duck typing works (in part) via the `isinstance()` and `issubclass()` built-in functions. 

These functions are used to determine class relationships. 

They rely on two internal methods that classes can provide: `__instancecheck__()` and `__subclasscheck__()`. 

An ABC class can provide a `__subclasshook__()` method, which is used by the `__subclasscheck__`()method to assert that a given class is a proper subclass of the abstract base class. 

The details are a bit beyond this book; consider this a signpost pointing out the path that needs to be followed when creating novel classes that need to live side by side with built-in classes.

# ABSTRACT BASE CLASSES and TYPE HINTS

The concept of an abstract base class is closely tied to the idea of a generic class.

An abstract base class is often generic with respect to some detail that is supplied by a concrete implementation.

Most of Python's generic classes – classes like list, dict, and set – can be used as type hints, and these hints can be parameterized to narrow the domain. 

There's a world of difference between `list[Any]` and `list[int]`; the value `["a", 42, 3.14]` is valid for the first type hint, but invalid for the other. 

This concept of parameterizing the generic type to make it more specific often applies to abstract classes, also.

For this to work, you will often need to incorporate `from __future__ import annotations` at the top of your module.

This is a new feature in Python 3.7 that allows you to use forward references in type hints.

This modifies the behaviour of Python to permit function and variable annotations to parameterize these standard collections.

**NOTE**

**Generic** classes and abstract classes are not the same thing. The two concepts overlap but are distinct.

Generic classes have an implicit relationship with `Any`. This often needs to be narrowed using type parameters, like `list[int]`.

The list class is concrete, and when we want to extend it, we will need to plug in a ccalss name to replace the `Any` type.

The Python interpreter does not use generic class hints in any way, they are only ckeced by static analysis tools such as mypy.

**Abstract** classes have placeholders instead of one or more methods. 

These placeholder methods require a design decision that supplies a concrete implementation.

These classes are not completely defined. When we extend it, we will need to provide a concrete method implementation.

If we do not provide the missing methods, the interpreter will raise a runtime exception when we try to create an instance of an abstract class.

Some classes can be both abstract and generic. As noted above, the type parameter helps mypy understand our intention but is not required. 

The concrete implementation is required.

Another concept that is adjacent to abstract classes is the **protocol**.

This is the essence of how duck typing works: when two classes have the same batch methods, they both adhere to a common protocol.

Any time we see classes with similar methods, there is a common protocol; this may be formalized with a type hint.

Consider objects that can be hashed.

Immutable classes implement the `__hash__()` method, including strings, integers and tuples.

Generally, mutable classes do not implement the `__hash__()` method, this includes lists, sets, and dictionaries.

If we attempt to write a type hint like `dict[list[int], list[str]]`, mypy will complain that the `list[int]` cannot be used as a key.

It cannot be a key because the given type, `list[int]`, does not implement the `Hashable` protocol.

At runtime, the attempt to create a dictionary item with a mutable key will fail for the same reason: a list does not implement the required method.



# THE collections.abc MODULE

One porminent use of abstract bassse classes is in the `collections.abc` module.

This module provides a bstract base calss definitions for Python's built in collections.

This is how `list`, `set` and `dict` (and a few other) can be built from individual component definitions.

We can use the definitions to build our own unique data structures in ways that overlap with built-in structures.

We can also use the definitions when we want to write a type hint for a specific feature of a data structure, without being overly specific about alternative implementations that might also be acceptable.

The definitions in the `collections.abc` do not include `list`, `set` and `dict`.

Instead, the module provides definitions like `MutableSequence`, `MutableSet`, and `MutableMapping`, which are abstract base classes for which `list`, `set` and `dict` classes we use are the concrete implementations.

Let's follow the various aspects of the definition of `Mapping` bact to their origins.

Python's built-in `dict` class is a concrete implementation of the `MutableMapping` abstract base class.

The abstraction comes from the idea of mapping a key to a value.

The `MutableMapping` class depends on the `Mapping` definition, an immutable, frozed dictionary, potentially optimized for lookups.

Let's follow the relationships among these abstractions.

Here's a diagram that shows the relationships among the various classes:

![MutableMapping](./MutableMapping.png)

Starting in the middle, we can see the `Mapping` definition depends on the `Collectionclass` definition. 

The definition of the Collection abstract class, in turn, depends on three other abstract base classes: `Sized`, `Iterable`, and `Container`. 

Each of these abstractions demands specific methods.

If we're going to create a lookup-only dictionary – a concrete `Mapping` implementation – we'll need to implement at least the following methods:

The `Sized` abstraction requires an implementation for the `__len__()` method. 

This lets an instance of our class respond to the `len()` function with a useful answer.


The `Iterable` abstraction requires an implementation for the `__iter__()` method.
 
This lets an object work with the for statement and the `iter()` function. 

The `Container` abstraction requires an implementation for the `__contains__() `method. 

This permits the in and not in operators to work.


The `Collection` abstraction combines Sized, Iterable, and Container without introducing additional abstract methods.

The `Mapping` abstraction, based on Collection, requires, among other things, `__getitem__()`, `__iter__()`, and` __len__()`. 

It has a default definition for `__contains__()`, based on whatever `__iter__()` method we provide. 

The Mapping definition will provide a few other methods, also.

This list of methods comes directly from the abstract relationships in the base classes. 

By building our new dictionary-like immutable class from these abstractions, we can be sure that our class will collaborate seamlessly with other Python generic classes.

When we look at the documentation in `https://docs.python.org/3.9/library/collections.abc.html`, we see abstract class definitions and the definitions they depend on.

There is a lattice of dependencies showing overlap among the class definitions.

It is this overlap that allows us to use a `for` statement to iterate through every kind of collection that implements the `Iterable` abstract base class.



Let's define out own immutable `Mapping` object implementation by extending the abstract classes.

The goal is to be able to load our dictionary-like mapping once with keys and values, and use it to map the keys to their values.

Since we are not going to allow any updates, we can apply a variety of algorithms to make it very fast as well as very compact. 

We are going to create a dictionary like mapping from some key to an object of any possible type.

We have defined the key with the type `Comparable` because we want to be able to compare the keys and sort them into order.

Searching through a list in order is often more efficient than searching through a list in random order.

We will look at the core of `Lookup` class definition first. We will returnt to the `Comparable` class definition after solidifying the essentials of a new kind of mapping from keys to values.

When we look at ways we can construct a dictionary, we see that a dictionary can be built two different kinds of data structures. 

The two data structures are exemplified by following:

In [169]:
x = dict({'a': 42, 'b': 7, 'c': 6})

y = dict([('a', 42), ('b', 7), ('c', 6)])

x == y

True

We can build a mapping from an existing mapping, or we can build a mapping from a sequence of two-tuples with keys and values.

This means there are two seperate definitions for the `__init__()` method.

1- `def __init__(self, source: BaseMapping) -> None:`

2- `def __init__(self, source: Iterable[tupple[Comparable, Any]]) -> None:`

These two definitions have distinct type hints.

To make it clear to mypy, we need to provide overloaded method definitions.

This is done with a special decoration from the `typing` module, `@overload`.

We'll provide two method definitions with the two alternatives; after these, we'll provide the real method definition that does the useful work. 

Because these are type hints, they're not required. They're wordy, and they help us be sure we've got a sensible implementation.

Here is the first part of the `Lookup` class definition.

We will break this into pieces because the `__init__()` method needs to cover these two cases defined by the alternative overloads:

In [170]:
from __future__ import annotations
from collections import abc 
from typing import Protocol, Any, overload, Union
import bisect
from typing import Iterator, Iterable, Sequence, Mapping 

from typing import Protocol, Any
class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool: ...
    def __ne__(self, other: Any) -> bool: ...
    def __le__(self, other: Any) -> bool: ...
    def __lt__(self, other: Any) -> bool: ...
    def __ge__(self, other: Any) -> bool: ...
    def __gt__(self, other: Any) -> bool: ...

def __len__(self) -> int:
        return len(self.key_list)
    
def __iter__(self) -> Iterator[Comparable]:
        return iter(self.key_list)
    
def __contains__(self, key: object) -> bool:
        index = bisect.bisect_left(self.key_list, key)
        return key == self.key_list[index]
    
def __getitem__(self, key: Comparable) -> Any:
        index = bisect.bisect_left(self.key_list, key)
        if key == self.key_list[index]:
            return self.value_list[index]
        raise KeyError(key)    


BaseMapping = abc.Mapping[Comparable, Any]
class Lookup(BaseMapping):
    @overload
    def __init__(
          self, 
          source: Iterable[tuple[Comparable, Any]]
    ) -> None:
        ...
    @overload
    def __init__(self, source: BaseMapping) -> None:
        ...
    def __init__(
          self, 
          source: Union[Iterable[              tuple[Comparable, Any]]
              BaseMapping, None] = None):
    
        sorted_pairs: Sequence[tuple[Comparable, Any]]
        if isinstance(source, Sequence):
            sorted_pairs = sorted(source)
        elif isinstance(source, abc.Mapping):
            sorted_pairs = sorted(source.items())
        else:
            sorted_pairs = []
        self.key_list = [p[0] for p in sorted_pairs]
        self.value_list = [p[1] for p in sorted_pairs]


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2079397366.py, line 46)

The `__init__()` method needs to handle three cases for loading a mapping. 

This means building the values from a sequence of pairs, or building the values from another mapping object, or creating an empty sequence of values. 

We need to separate the keys from the values and put them into two parallel lists. 

A sorted list of keys can be rapidly searched to find a match. 

The sorted list of values is returned when we get a key's value from the mapping.

Here are the imports needed:

from `__future__` import annotations
from collections import abc
from typing import Protocol, Any, overload, Union
import bisect
from typing import Iterator, Iterable, Sequence, Mapping

Here are the other abstract methods that are defined by the @abstractmethod decorator. We provide the following concrete implementations:


In [179]:
    def __len__(self) -> int:
        return len(self.key_list)
    
    def __iter__(self) -> Iterator[Comparable]:
        return iter(self.key_list)
    
    def __contains__(self, key: object) -> bool:
        index = bisect.bisect_left(self.key_list, key)
        return key == self.key_list[index]
    
    def __getitem__(self, key: Comparable) -> Any:
        index = bisect.bisect_left(self.key_list, key)
        if key == self.key_list[index]:
            return self.value_list[index]
        raise KeyError(key)


NameError: name 'Iterator' is not defined

The `__len__()`, `__iter__()`, and `__contains__`() methods are required by the Sized, Iterable, and Container abstract classes. 

The Collection abstract class combines the other three without introducing any new abstract methods.

The `__getitem__()` is required to be a Mapping. Without it, we can't retrieve an individual value for a given key.

The use of the bisect module is one way to find a specific value rapidly in a sorted list of keys. 

The `bisect.bisect_left()` function finds the spot where a key belongs in a list. 

If the key is there, we can return the value to which it maps. If the key is not there, we can raise the KeyError exception.

Note that the `__contains__()` definition has the object class as the type hint, unlike the other methods. 

This is required because Python's in operation needs to support any kind of object, even ones that don't obviously support the Comparable protocol.

------------------------------------------------

The general approach to using abstract classes is this:

1- Find a class that does most of what you need.


2- Identify the methods in the collections.abc definitions that are marked as abstract. The documentation often gives a lot of information, but you'll also have to look at the source.

3- Subclass the abstract class, filling in the missing methods.

4- While it can help to make a checklist of the methods, there are tools to help with this. Creating a unit test means you need to create an instance of your new class. If you haven't defined all the abstract methods, this will raise an exception. Using mypy will also spot abstract methods that aren't properly defined in the concrete subclass.


This is a powerful way to reuse code when we choose the abstractions well; a person can form a mental model of the class without knowing all of the details. 

It's also a powerful way to create closely related classes that can easily be examined by mypy. 

Beyond those two advantages, the formality of marking a method as abstract gives us a runtime assurance that the concrete subclass really does implement all the required methods.

Now that we've seen how to use an abstract base class, let's look at defining a new abstraction.

# CREATING YOUR OWN ABSTRACT BASE CLASS

We have two general paths to create classes.

'e can leverage duck typing or we can define common abstractions. 

When we leverage duck typing, we can formalize the related types by creating a type hint using a protocol definition to enumerate the common methods, or a `Union[]` to enumerate the common types.

There are an almost unlimited number of influencing factors that suggest one or the other approach. 

While duck typing offers the most flexibility, we may sacrifice the ability to use mypy. 

An abstract base class definition can be wordy and potentially confusing.

We'll tackle a small problem. 

We want to build a simulation of games that involve polyhedral dice. 

These are the dice including four, six, eight, twelve, and twenty sides. The six-sided dice are conventional cubes. Some sets of dice include 10-sided dice, which are cool, but aren't – technically – a regular polyhedron; they're two sets of five "kite-shaped" faces.

One question that comes up is how best to simulate rolls of these different shaped dice. 

There are three readily available sources of random data in Python: the random module, the os module, and the secrets module. 

If we turn to third-party modules, we can add in cryptographic libraries like pynacl, which offer yet more random number capabilities.

Rather than bake the choice of random number generator into a class, we can define an abstract class that has the general features of a die. 

A concrete subclass can supply the missing randomization capability. 

The random module has a very flexible generator. 

The os module's capability is limited, but involves using an entropy collector to increase randomness. Flexibility and high entropy are generally combined by cryptographic generators.

To create our dice-rolling abstraction, we'll need the abc module. 

This is distinct from the collections.abc module. The abc module has the foundational definitions for abstract classes:

In [191]:
import abc
import random

class Die(abc.ABC):
  
    def __init__(self) -> None:
        self.face: int
        self.roll()
        
    @abc.abstractmethod
    def roll(self) -> None:
        ...
    
    def __repr__(self) -> str:
        return f"{self.face}"

We've defined a class that inherits from the abc.ABC class. 

*Using ABC as the parent class assures us that any attempt to create an instance of the Die class directly will raise a TypeError exception.* 

This is a runtime exception; it's also checked by mypy.

We've marked a method, roll(), as abstract with the @abc.abstract decorator. 

This isn't a very complex method, but any subclass should match this abstract definition. This is only checked by mypy. 

Of course, if we make a mess of the concrete implementation, things are likely to break at runtime. Consider this mess of code:

class Bad(Dice):
    def roll(self, a:int , b: int) -> float:
        return (a+b)/2
        
This will raise a TypeError exception at runtime. 

The problem is caused by the base class `__init__()` not providing the a and b parameters to this strange-looking roll() method. 

This is valid Python code, but it doesn't make sense in this context. 

The method will also generate mypy errors, providing ample warning the method definition doesn't match the abstraction.

Here's what two proper extensions to the Die class look like:

In [192]:
class D4(Die):
    def roll(self) -> None:
        self.face = random.choice(1, 2, 3, 4)
        
class D6(Die):
    def roll(self) -> None:
        self.face = random.randint(1, 6)

We've provided methods that provide a suitable definition for the abstract placeholder in the Die class. 

They use vastly different approaches to selecting a random value. 

The four-sided die uses random.choice(). 

The six-sided die – the common cube most people know – uses random.randint().

Let's go a step further and create another abstract class. This one will represent a handful of dice. 

Again, we have a number of candidate solutions, and we can use an abstract class to defer the final design choices.

The interesting part of this design is the differences in the rules for games with handfuls of dice. 

In some games, the rules require the player to roll all the dice. 

The rules for a lot of games with two dice require the player to roll both dice. 

In other games, the rules allow players to save dice, and re-roll selected dice. I

n some games, like Yacht, the players are allowed at most two re-rolls. 

In other games, like Zilch, they are allowed to re-roll until they elect to save their score or roll something invalid and lose all their points, scoring zilch (hence the name).

These are dramatically different rules that apply to a simple list of Die instances. Here's a class that leaves the roll implementation as an abstraction:

In [193]:
from typing import Type
import abc
import random

class Dice(abc.ABC):
    
    def __init__(self, n: int, die_class: Type[Die]) -> None:
        self.dice = [die_class() for _ in range(n)]
    @abc.abstractmethod
    def roll(self) -> None:
        ...
    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice)


The `__init__()` method expects an integer, n, and the class used to create Die instances, named die_class. 

The type hint is `Type[Die]`, telling mypy to be on the lookout for any subclass of the abstract base class Die. 

We don't expect an instance of any of the Die subclasses; we expect the class object itself. 

We'd expect to see SomeDice(6, D6) to create a list of six instances of the D6 class.

We've defined the collection of Die instances as a list because that seems simple. Some games will identify dice by their position when saving some dice and rerolling the remainder of the dice, and the integer list indices seem useful for that.

This subclass implements the roll-all-the-dice rule:

In [194]:
class SimpleDice(Dice):
    def roll(self) -> None:
        for d in self.dice:
            d.roll()


Each time the application evaluates roll(), all the dice are updated.

In [195]:
sd = SimpleDice(6, D6)
sd.roll()

In [196]:
sd.total

17

Here's another subclass that provides a dramatically different set of methods. Some of these fill in the spaces left by abstract methods. Others, however, are unique to the subclass:

In [197]:


from typing import Set, Iterable


class YachtDice(Dice):
    def __init__(self) -> None:
        super().__init__(5, D6)
        self.saved: Set[int] = set()
    def saving(self, positions: Iterable[int]) -> "YachtDice":
        if not all(0 <= n < 6 for n in positions):
            raise ValueError("Invalid position")
        self.saved = set(positions)
        return self
    def roll(self) -> None:
        for n, d in enumerate(self.dice):
            if n not in self.saved:
                d.roll()
        self.saved = set()


We've created a set of saved positions. This is initially empty. 

We can use the saving() method to provide an iterable collection of integers as positions to save. It works like this:

In [198]:
sd = YachtDice()

sd.roll()


In [199]:
sd.dice

[1, 5, 5, 1, 5]

In [200]:
sd.saving([0, 2, 4]).roll()

In [201]:
sd.dice

[1, 1, 5, 1, 5]

In both cases, the Die class and the Dice class, it's not clear that the abc.

ABC base class and the presence of an `@abc.abstractmethod` decoration is dramatically better than providing a concrete base class with a common set of default definitions.

In some languages, the abstraction-based definition is required. 

In Python, because of duck typing, abstraction is optional. 

In cases where it clarifies the design intent, use it. 

In cases where it seems fussy and little more than overhead, set it aside.

Because it's used to define the collections, we'll often use the `collection.abc` names in type hints to describe the protocols objects must follow. 

In less common cases, we'll leverage the collections.abc abstractions to create our own unique collections.

# DEMYSTIFYING the MAGIC

We have used abstract base classes and it is clear they are doing a lot of work.

Let's look inside the class to see some of what's going on:

from dice import Die

Die.__abstractmethods__
frozenset({'roll'})


Die.roll.__isabstractmethod__
True


Abstract method, `roll()`, is tracked in a specially named attribute, `__abstractmethods__` of the class.

This suggests what the `@abc.abstractmethod` decorator is doing.

This decorator sets `__isabstractmethod__` to mark the method.

When Python finally builds the class from the various methods and attributes, the list of abstractions is also collected to create a class-level set of moethods that must be implemented.

Any subclass that extends `Die` will also inherit this `__abstractmethods__` set.

When methods are defined inside the subclass, names are removed from the set as Python builds the class from the definitions.

We can only create instances of a class where the set of abstract methods in the class is empty.

Central to this is the way classes are created: 

**A class builds objects.**

This is the essence of most of object-oriented programming. But what is a class?

- A class is another object with two very limited jobs: it has the special methods used to create and manage instances of the class, and it also acts as a container for the method definitions for objects of the class

- We think of building class objects with the `class` statement, which leaves open the question of how the `class ` statement builds the `class` object.

- The `type` class is the internal object that builds our application classes. When we enter the code for a class, the details of construction are actually the responsibility of methods of the `type` class.

- After `type` has created our application class, our class then creates the application objects that solve our problem.

**The `type` object is called the metaclass, the class used to build classes.**

This means every class object is an instance of `type`.

Most of the time, we are perfectly happy with letting a class statement be handled bt the `type` metaclass, so our application code can run.

There is one place, however, where we might want to change how `type` works.

Because `type` is itself a class, it can be extended.

A class `abc.ABCMeta` extends the `type` class to check for methods decorated with `@abstractmethod`.

When we extend `abc.ABC`, we are creating a new class that uses the `ABCMeta` metaclass.

We can see this in the value of the special `__mro__` attribute of the `ABCMeta` class; this attribute lists the classes used for resolving method names **(MRO is Method Resolution Order)**. 

This special attribute lists the following classes to be searched for a given attribute: the `abc.ABCMeta` class, the type class, and finally the object class.

We can use the ABCMeta metaclass explicitly when we create a new class, if we want:


In [203]:
class DieM(metaclass=abc.ABCMeta):
    def __init__(self) -> None:
        self.face: int
        self.roll()
    @abc.abstractmethod
    def roll(self) -> None:
        ...


Now that we've seen how classes are built, we can consider other things we can do when creating and extending classes. 

Python exposes the binding between the syntactic operators, like the `/` operator, and the methods of the implementing class. 

This allows the `float` and `int` classes to do different things with the `/` operator, but it can also be used for quite different purposes. 

# OPERATOR OVERLOADING

Python's operators `+`, `/`, `-`, `*`, and so on, are implemented by special methods on classes.

We can apply Python operators more widely than the built-in numbers and collection types.

Doing this can be called **overloading** the operators: letting them work with more than  the build-in types.

Looking back at the The `collections.abc` module section, earlier in this chapter, we dropped a hint about how Python connects some built-in features with our classes. 

When we look at the `collections.abc.Collection` class, it is the abstract base class for all `Sized`, `Iterable`, `Containers`; it requires three methods that enable two built-in functions and one built-in operator:

- `__len()__` method is used by the built-in len() function.

- -The `__iter__()` method is used by the built-in `iter()` function, which means it's used by the `for` statement.

- The `__contains__()` method is used by the built-in `in` operator. This operator is implemented by methods of built-in classes.

It's not wrong to imagine the built-in len() function has this definition:

In [205]:
from typing import Sized


def len(object: Sized) -> int:
    return object.__len__()


When we ask for `len(x)`, it's doing the same thing as `x.__len__()`, but is shorter, easier to read, and easier to remember. 

Similarly, `iter(y)` is effectively `y.__iter__()`. And an expression like `z in S` is evaluated as if it was `S.__contains__(z)`.

And yes, with a few exceptions, all of Python works this way. 

We write pleasant, easy-to-read expressions that are transformed into special methods. 

**The only exceptions are the logic operations: and, or, not, and if-else. These don't map directly to special method definitions**.

Because almost all of Python relies on the special methods, it means we can change their behavior to add features. 

We can overload the operators with new data types. 

One prominent example of this is in the pathlib module:

    from pathlib import Path
    home = Path.home()
    home / "miniconda3" / "envs"
PosixPath('/Users/slott/miniconda3/envs')


What does not vary is that the `/` operator is used to connect a `Path` object with string objects to create a new `Path` object.

The `/` operator is implemented bt the `__truediv__()` and `__rtruediv__()`methods.

In order to make operations commutative, Python has two places to look for an implementation.

Given an expression of `A` op `B`, where op is any of the Python operators like `__add__` for `+`, Python does the following checks for special methods to implement the operator:

1- There is a special case when `B` is proper subclass of `A`. 

In those rare cases, the order is reversed so `B.__r` op `__(A)` can be tried before any of the others. This lets the subclass `B` override an operation from superclass `A`.

2- Try `A.__` op `__(B)`. If this returns a value that's not the special `NotImplemented` value, this is the result. 

For a Path object expression like `home / "miniconda3"`, this is effectively `home.__truediv__("miniconda3")`. 

A new Path object is built from the old Path object and the string.

3- Try `B.__r` op `__(A)`. This might ne the `__radd__()` method for the reverse addition implementation.

If this method returns a value other than `NotImplemented` value, this is the result.

Note that the operand ordering is reversed.  

For commutative operations, like addition and multiplication, this does not matter. 

For non-commutative operations, like subtraction and division, the change in ordering needs to be reflected in the implementation.

Let's return to our handful of dice example. 

We can implement a `+` operator to add a `Die` instance to a collection of `Dice`. 

We'll start with a base definition of a class that contains a heterogenous handful of different kinds of dice. 

Check the previous `Dice` class, which assumed homogenous dice. 

This isn't an abstract class; it has a definition of `roll` that re-rolls all the dice. 

We'll start with some basics and then incorporate the `__add__()` special method:

In [210]:
from typing import Type, Any
import abc


class DDice:
    def __init__(self, *die_class: Type[Die]) -> None:
        self.dice = [dc() for dc in die_class]
        self.adjust: int = 0
    def plus(self, adjust: int = 0) -> "DDice":
        self.adjust = adjust
        return self
    def roll(self) -> None:
        for d in self.dice:
            d.roll()
    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice) + self.adjust


This shouldn't be much of a surprise. It looks a lot like the `Dice` class defined above. 

We've added an `adjust` attribute set by the `plus()` method so we can use DDice(D6, D6, D6).plus(2). 

It fits better with some tabletop role-playing games (TTRPGs).

Also, recall that we provide the types of the dice to the DDice class, not instances of the dice. 

We use the class object, D6, not a Die instance, created by an expression like D6(). 

The instances of the classes are created by DDice in the `__init__()` method.

Here's the cool part: we can use the plus operator with DDice objects, Die classes, and integers to define a complex roll of the dice:

In [211]:
def __add__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and issubclass(die_class, Die):
        new_classes = [type(d) for d in self.dice] + [die_class]
        new = DDice(*new_classes).plus(self.adjust)
        return new
    elif isinstance(die_class, int):
        new_classes = [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(die_class)
        return new
    else:
        return NotImplemented
def __radd__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and issubclass(die_class, Die):
        new_classes = [die_class] + [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(self.adjust)
        return new
    elif isinstance(die_class, int):
        new_classes = [type(d) for d in self.dice]
        new = DDice(*new_classes).plus(die_class)
        return new
    else:
        return NotImplemented


These two methods are similar in many ways. We check for three separate kinds of `+` operations:

If the argument value, `die_class`, is a type, and it's a subclass of the `Die` class, then we're adding another `Die` object to a `DDice` collection. 

It's an expression like DDice(D6) + D6 + D6. The semantics of most operator implementations is to create a new object from the previous objects.

If the argument value is an integer, then we're adding an adjustment to a set of dice. 

This is something like DDice(D6, D6, D6) + 2.

If the argument value is neither a subclass of `Die` nor an integer, then something else is going on, and this class doesn't have an implementation. 

This may be some kind of bug, or it might be that the other class involved in the operation can provide an implementation; returning NotImplemented gives the other object a chance at performing the operation.


Because we've provided `__radd__()` as well as `__add__()`, these operations are commutative. We can use expressions like D6 + DDice(D6) + D6 and 2 + DDice(D6, D6).

We need to make specific `isinstance()` checks because Python operators are completely generic, and the expected type hint must be `Any`. 

We can only narrow down the applicable types through runtime checks. 

The mypy program is astute about following the branching logic to confirm that an integer object was properly used in an integer context.

"But wait," you say. "My favorite game has rules that call for 3d6+2." This is shorthand for rolling three six-sided dice and adding two to the result. 

In many TTRPGs, this kind of abbreviation is used to summarize the dice.

Can we add multiplication to do this? There's no reason why not. 

For multiplication, we only need to worry about integers. D6 * D6 isn't used in any of the rules, but 3*D6matches the text of most TTRPG rules nicely:

In [213]:
def __mul__(self, n: Any) -> "DDice":
    if isinstance(n, int):
        new_classes = [type(d) for d in self.dice for _ in range(n)]
        return DDice(*new_classes).plus(self.adjust)
    else:
        return NotImplemented
def __rmul__(self, n: Any) -> "DDice":
    if isinstance(n, int):
        new_classes = [type(d) for d in self.dice for _ in range(n)]
        return DDice(*new_classes).plus(self.adjust)
    else:
        return NotImplemented


These two methods follow a similar design pattern to the` __add__()` and `__radd__()` methods. 

For each existing Die subclass, we'll create several instances of the class.
 
This lets us use `3 * DDice(D6) + 2` as an expression to define a dice-rolling rule. 

The Python operator precedence rules still apply, so the 3 * DDice(D6) portion is evaluated first.

Python's use of the various `__op__()` and `__rop__()` methods works out extremely well for applying the various operators to objects that are immutable: strings, numbers, and tuples being the primary examples. 

Our handful of dice presents a bit of a head-scratcher because the state of the individual dice can change. 

What's important is that we treat the composition of the hand as immutable. 

Each operation on a DDice object creates a new DDice instance.

What about mutable objects? 

When we write an assignment statement like `some_list += [some_item]`, we're mutating the value of the `some_list` object. 

The `+=` statement does the same thing as the more complex expression `some_list.extend([some_item])`. 

Python supports this with operators with names like `__iadd__()` and `__imul__()`. 

These are "in-place" operations, designed to mutate objects.

For example, consider:

y = DDice(D6, D6)

y += D6

This can be processed one of two ways:

If DDice implements `__iadd__()`, this becomes `y.__iadd__(D6)`. The object can mutate itself in place.

If DDice does not implement `__iadd__()`, this is `y = y.__add__(D6)`. 

The object creates a new, immutable object, and that's given the old object's variable name. 

This lets us do things like `string_variable += "."`. 

Under the hood, string_variable is not mutated; it's replaced.

If it makes sense for an object to be mutable, we can support in-place mutation of a DDice object with this method:


In [216]:
def __iadd__(self, die_class: Any) -> "DDice":
    if isinstance(die_class, type) and issubclass(die_class, Die):
        self.dice += [die_class()]
        return self
    elif isinstance(die_class, int):
        self.adjust += die_class
        return self
    else:
        return NotImplemented


The __iadd__() method appends to the internal collection of dice. 

It follows rules similar to the __add__() methods: when a class is provided, an instance is created, and it's added to the `self.dice `list; if an integer is provided, it's added to the `self.adjust` value.

We can now perform incremental changes to a single dice-rolling rule. 

We can mutate the state of a single DDice object using assignment statements. 

Because the object mutates, we aren't creating a lot of copies of the object. The creation of complex dice looks like this:

 y = DDice(D6, D6)
 y += D6
 y += 2


This builds the 3d6+2 dice roller in incremental pieces.

The use of the internal special method names allows for seamless integration with other Python features. 

We can build classes using collections.abc that fit with existing collections. 

We can override the methods implementing the Python operators to create easy-to-use syntax.

We can leverage the special method names to add features to Python's built-in generic collections.

# EXTENDING THE BUILT-IN 

